### Importing the necessary libraries 

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Checking for files in the required location
import os
for dirname, _, filenames in os.walk('/Users/priyanuj/CampusX_ML_Course'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/Users/priyanuj/CampusX_ML_Course/ridge_regression_from_scratch_ols.ipynb
/Users/priyanuj/CampusX_ML_Course/Multiple_Linear_Regression_0520.ipynb
/Users/priyanuj/CampusX_ML_Course/train_digit.csv
/Users/priyanuj/CampusX_ML_Course/Titanic_Survival_Analysis_10212023.ipynb
/Users/priyanuj/CampusX_ML_Course/ridge_regression_from_scratch_gradient_descent_methodology.ipynb
/Users/priyanuj/CampusX_ML_Course/Gradient_Descent_function_video_1_first_code.ipynb
/Users/priyanuj/CampusX_ML_Course/test.csv
/Users/priyanuj/CampusX_ML_Course/decision_tree_clf_from_scratch.ipynb
/Users/priyanuj/CampusX_ML_Course/PCA_102123.ipynb
/Users/priyanuj/CampusX_ML_Course/perceptron_trick_code_classification.ipynb
/Users/priyanuj/CampusX_ML_Course/kNNClassifier_from_scratch_1012.ipynb
/Users/priyanuj/CampusX_ML_Course/logistic_regression_from_scratch.ipynb
/Users/priyanuj/CampusX_ML_Course/train.csv
/Users/priyanuj/CampusX_ML_Course/test_digit.csv
/Users/priyanuj/CampusX_ML_Course/train_data.csv
/Users/priyanuj/

### Loading the dataset

In [3]:
#Importing titanic data from the required location
input_ads_pre = pd.read_csv('/Users/priyanuj/CampusX_ML_Course/train_data.csv')
input_ads_pre.drop(columns=['Unnamed: 0','Title_1','Title_2','Title_3','Title_4'],inplace=True)
#----------------------------------------------------------------------------------------------
print(input_ads_pre.shape)
input_ads_pre.head()

(792, 12)


,PassengerId,Survived,Sex,Age,Fare,Pclass_1,Pclass_2,Pclass_3,Family_size,Emb_1,Emb_2,Emb_3
0,1,0,1,0.2750,0.014151,0,0,1,0.1,0,0,1
1,2,1,0,0.4750,0.139136,1,0,0,0.1,1,0,0
2,3,1,0,0.3250,0.015469,0,0,1,0.0,0,0,1
3,4,1,0,0.4375,0.103644,1,0,0,0.1,0,0,1
4,5,0,1,0.4375,0.015713,0,0,1,0.0,0,0,1


### Basic Exploratory Data Analysis

In [5]:
# Total survived vs not-survived count/split in the training data
input_ads_pre['Survived'].value_counts()

0    486
1    306
Name: Survived, dtype: int64

In [6]:
pd.DataFrame(input_ads_pre.isnull().sum()).T

,PassengerId,Survived,Sex,Age,Fare,Pclass_1,Pclass_2,Pclass_3,Family_size,Emb_1,Emb_2,Emb_3
0,0,0,0,0,0,0,0,0,0,0,0,0


In [7]:
input_ads_pre.describe()

,PassengerId,Survived,Sex,Age,Fare,Pclass_1,Pclass_2,Pclass_3,Family_size,Emb_1,Emb_2,Emb_3
count,792.000000,792.000000,792.000000,792.000000,792.000000,792.000000,792.000000,792.000000,792.000000,792.000000,792.000000,792.000000
mean,396.500000,0.386364,0.647727,0.368244,0.064677,0.243687,0.208333,0.547980,0.088636,0.185606,0.092172,0.720960
std,228.774999,0.487223,0.477980,0.162994,0.100987,0.429577,0.406373,0.498007,0.154485,0.389034,0.289451,0.448811
min,1.000000,0.000000,0.000000,0.008375,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,198.750000,0.000000,0.000000,0.275000,0.015469,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,396.500000,0.000000,1.000000,0.350000,0.028302,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000
75%,594.250000,1.000000,1.000000,0.437500,0.061045,0.000000,0.000000,1.000000,0.100000,0.000000,0.000000,1.000000
max,792.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### Shuffling the ADS

In [8]:
from sklearn.utils import shuffle

input_ads = shuffle(input_ads_pre, random_state = 100)
input_ads.reset_index(drop=True, inplace = True)
print(input_ads.shape)
input_ads.head()

(792, 12)


,PassengerId,Survived,Sex,Age,Fare,Pclass_1,Pclass_2,Pclass_3,Family_size,Emb_1,Emb_2,Emb_3
0,371,1,1,0.3125,0.108215,1,0,0,0.1,1,0,0
1,556,0,1,0.7750,0.051822,1,0,0,0.0,0,0,1
2,624,0,1,0.2625,0.015330,0,0,1,0.0,0,0,1
3,98,1,1,0.2875,0.123667,1,0,0,0.1,1,0,0
4,144,0,1,0.2375,0.013175,0,0,1,0.0,0,1,0


### Manipulation of data into train and test

In [9]:
target = 'Survived' #To predict

#--------------------------------------------------------------------------------
#Splitting into X & Y datasets (supervised training)
X = input_ads[[cols for cols in list(input_ads.columns) if target not in cols]]
y = input_ads[target]

#--------------------------------------------------------------------------------
#Since test data is already placed in the input folder separately, we will just import it
test_ads_pre = pd.read_csv('/Users/priyanuj/CampusX_ML_Course/test_data.csv')
test_ads_pre.drop(columns=['Unnamed: 0','Title_1','Title_2','Title_3','Title_4'],inplace=True) #Dropping un-necessary columns
test_ads = shuffle(test_ads_pre,random_state=100)
test_ads = test_ads.reset_index(drop=True)

#Splitting into X & Y datasets (supervised training)
X_test = test_ads[[cols for cols in list(test_ads.columns) if target not in cols]]
y_test = test_ads[target]

print('Train % of total data:',100 * X.shape[0]/(X.shape[0] + X_test.shape[0]))
#--------------------------------------------------------------------------------
#Manipulation of datasets for convenience and consistency
X_arr = np.array(X)
X_test_arr = np.array(X_test)

y_arr = np.array(y).reshape(X_arr.shape[0],1)
y_test_arr = np.array(y_test).reshape(X_test_arr.shape[0],1)

#--------------------------------------------------------------------------------
#Basic Summary
print(X_arr.shape)
print(X_test_arr.shape)
print(y_arr.shape)

Train % of total data: 88.78923766816143
(792, 11)
(100, 11)
(792, 1)


### Independent Logistic Regression and Decision Tree model

In [10]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn import tree
#max_depth=2,min_samples_split=7,min_samples_leaf=8,

#Decision Tree Classifier
dt_clf = DecisionTreeClassifier(random_state=100)
dt_clf.fit(X_arr,y_arr)
sklearn_preds_dt = dt_clf.predict(X_test_arr)

#---------------------------------------------------------------------------
#Logistic Regression
lr_clf = LogisticRegression(solver='sag',random_state=100)
lr_clf.fit(X_arr,y_arr)
sklearn_preds_lr = lr_clf.predict(X_test_arr)

#Evaluation of the predictions
print('#- Decision Tree ---------------------------------------------------#')
print('ROC AUC of test set :',roc_auc_score(y_test_arr,sklearn_preds_dt))
print('Accuracy of test set :',accuracy_score(y_test_arr,sklearn_preds_dt),'\n')

print('#- Logistic Regression ---------------------------------------------#')
print('ROC AUC of test set :',roc_auc_score(y_test_arr,sklearn_preds_lr))
print('Accuracy of test set :',accuracy_score(y_test_arr,sklearn_preds_lr),'\n')

#- Decision Tree ---------------------------------------------------#
ROC AUC of test set : 0.7986111111111112
Accuracy of test set : 0.82 

#- Logistic Regression ---------------------------------------------#
ROC AUC of test set : 0.5
Accuracy of test set : 0.64 



## Bagging Logic from scratch

### UDF for bootstrap sampling

In [26]:
#UDF for bootstrapping sampling logic
def bootstrapped_sample(arr,random_state):
    
    np.random.seed(random_state)
    # The function is mentioned such as way that, with replacement, the total number of rows will be equal to the 
    # number of rows in the array/ dataset provided
    boot_sample_idx = np.random.choice(a=range(len(arr)),size=len(arr),replace=True)
    boot_sample = arr[boot_sample_idx]
    
    return boot_sample

### UDF for bagging (will work similarly for regression as well)

In [55]:
def bagging_ensemble_clf(n_estimators, estimator, X_arr_, y_arr_, test, threshold = 0.5):
    
    pred_array = np.array([np.nan]*len(test)).reshape((len(test)),1)
    
    joint_arr = np.append(X_arr_, y_arr_, axis = -1)
    
    # Bootstrapping and building the model iteratively
    
    for n in range(n_estimators):
        # using random state = n gives more randomness per iteration for selection of points
        sample = bootstrapped_sample(joint_arr, random_state = n)
        
        X_sample = sample[:,0:-1]
        y_sample = sample[:,-1]
        
        #fitting the estimator
        estimator.fit(X_sample, y_sample)
        
        pred_array_temp = np.array(estimator.predict(test)).reshape((len(test)),1)
        # adding the predictions from the subsequent estimators to the array as columns
        pred_array = np.append(pred_array, pred_array_temp, axis = -1)
        
        
    # Aggregation
    # Removing the first column which contains nan values as defined in the first snippet
    pred_array = pred_array[:,1:]
    # aggregating the predictions from the estimators
    pred = np.sum(pred_array, axis = 1)
    
    n_preds = pred_array.shape[1] #Total number of columns
    
    pred = pred/n_preds
    
    pred = (pred>threshold).astype(int)
    print('Unique Preditions: ', np.unique(pred))
    
    return pred

### Initiating the bagging udf with decision tree model

In [56]:
decision_tree = DecisionTreeClassifier(random_state=100)

preds_dt = bagging_ensemble_clf(n_estimators=500, estimator=decision_tree, X_arr_=X_arr, y_arr_=y_arr, test=X_test_arr)
print(preds_dt.shape)

print('#- Manual Bagging w/ Decision Tree ---------------------------------------------------#')
print('ROC AUC of test set :',roc_auc_score(y_test_arr,preds_dt))
print('Accuracy of test set :',accuracy_score(y_test_arr,preds_dt),'\n')

Unique Preditions:  [0 1]
(100,)
#- Manual Bagging w/ Decision Tree ---------------------------------------------------#
ROC AUC of test set : 0.8081597222222222
Accuracy of test set : 0.84 



# Initiating the bagging udf with logistic regression model

In [57]:
log_reg = LogisticRegression(solver='saga',random_state=100)

preds_lr = bagging_ensemble_clf(n_estimators=500, estimator=log_reg, X_arr_=X_arr, y_arr_=y_arr, test=X_test_arr)
print(preds_lr.shape)

print('#- Manual Bagging w/ Logistic Regression Tree ---------------------------------------------------#')
print('ROC AUC of test set :',roc_auc_score(y_test_arr,preds_lr))
print('Accuracy of test set :',accuracy_score(y_test_arr,preds_lr),'\n')

Unique Preditions:  [0]
(100,)
#- Manual Bagging w/ Logistic Regression Tree ---------------------------------------------------#
ROC AUC of test set : 0.5
Accuracy of test set : 0.64 



### Random Forest Classifier - can have only decision trees and needs sampling of columns too at each node hence will use max_featuers. = 'sqrt' at each node for column splitting with replacement

In [58]:
decision_tree_with_col_sampling = DecisionTreeClassifier(max_features='sqrt', random_state=100)

preds_rf = bagging_ensemble_clf(n_estimators=500, estimator=decision_tree_with_col_sampling, X_arr_=X_arr, y_arr_=y_arr, test=X_test_arr)
print(preds_rf.shape)

print('#- Manual Random Forest ---------------------------------------------------#')
print('ROC AUC of test set :',roc_auc_score(y_test_arr,preds_dt))
print('Accuracy of test set :',accuracy_score(y_test_arr,preds_dt),'\n')

Unique Preditions:  [0 1]
(100,)
#- Manual Random Forest ---------------------------------------------------#
ROC AUC of test set : 0.8081597222222222
Accuracy of test set : 0.84 



## Sklearn Benchmarking

### Bagging with Decision Tree

In [59]:
from sklearn.ensemble import BaggingClassifier

decision_tree = DecisionTreeClassifier(random_state=100)

bagging_skl = BaggingClassifier(base_estimator=decision_tree,
                                n_estimators=500,
                                max_features=1.0,
                                bootstrap=True,
                                random_state=100,
                                n_jobs=-1)
bagging_skl.fit(X_arr,y_arr)
bagging_skl_pred = bagging_skl.predict(X_test_arr)

#--------------------------------------------------------------------------------------------------------
print('#- Sklearn Bagging Classifier ---------------------------------------------------#')
print('ROC AUC of test set :',roc_auc_score(y_test_arr,bagging_skl_pred))
print('Accuracy of test set :',accuracy_score(y_test_arr,bagging_skl_pred),'\n')

#- Sklearn Bagging Classifier ---------------------------------------------------#
ROC AUC of test set : 0.8081597222222222
Accuracy of test set : 0.84 



### Sklearn RandomForest

In [60]:
from sklearn.ensemble import RandomForestClassifier

rf_skl = RandomForestClassifier(n_estimators=500,
                                max_features='sqrt',
                                bootstrap=True,
                                random_state=100,
                                n_jobs=-1)
rf_skl.fit(X_arr,y_arr)
rf_skl_pred = rf_skl.predict(X_test_arr)

#--------------------------------------------------------------------------------------------------------
print('#- Sklearn Bagging Classifier ---------------------------------------------------#')
print('ROC AUC of test set :',roc_auc_score(y_test_arr,rf_skl_pred))
print('Accuracy of test set :',accuracy_score(y_test_arr,rf_skl_pred),'\n')

#- Sklearn Bagging Classifier ---------------------------------------------------#
ROC AUC of test set : 0.828125
Accuracy of test set : 0.85 



#### Insights : The manual implementations are giving almost identical quality of predictions for normal bagging and random forest with the sklearn versions

### END